# Model Experimentation
This notebook contains a minimal template for running ML experiments. 

## Imports and Setup

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Locations

In [12]:
data_dir = "../data/"
input_file = data_dir + '20200422_dataset.csv'
output_dir = "../outputs/"

images_dir = data_dir + 'images/'
indices_dir = data_dir + 'indices/'
pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'

areas = ['maicao', 'riohacha', 'uribia', 'arauca', 'cucuta', 'arauquita', 'tibu']
value_codes = {value : key.capitalize() for (key,value) in model_utils.VALUE_CODES.items()}

## Load Dataset

In [3]:
data = pd.read_csv(input_file).reset_index(drop=True) 
class_dist = data['target'].value_counts().rename(index=value_codes)

print('Data dimensions: {}'.format(data.shape))
print('\nClass distribution:\n{}\n'.format(class_dist))
data.head()

Data dimensions: (965034, 113)

Class distribution:
Unoccupied land        813575
Formal settlement      130162
Informal settlement     21297
Name: target, dtype: int64



,B1_2015-2016,B2_2015-2016,B3_2015-2016,B4_2015-2016,B5_2015-2016,B6_2015-2016,B7_2015-2016,B8_2015-2016,B9_2015-2016,B10_2015-2016,...,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira,target,area
0,0.15100,0.13225,0.14240,0.16430,0.18990,0.25700,0.2960,0.25510,0.32295,0.0396,...,-0.509745,0.105128,0.239614,0.449106,-0.718433,-0.042537,0.901237,1,3,0
1,0.15100,0.12885,0.13790,0.16185,0.18990,0.25700,0.2960,0.25755,0.32295,0.0396,...,-0.507485,0.105128,0.247826,0.464498,-0.716955,-0.039760,0.911490,1,3,0
2,0.15895,0.13730,0.15185,0.18915,0.22640,0.28555,0.3268,0.28085,0.35740,0.0416,...,-0.524371,0.073259,0.262348,0.446475,-0.722188,-0.033995,0.875915,1,3,0
3,0.15895,0.14630,0.17710,0.24240,0.22640,0.28555,0.3268,0.30980,0.35740,0.0416,...,-0.475631,0.073259,0.308045,0.524245,-0.689591,-0.019520,0.905289,1,3,0
4,0.15895,0.15345,0.19200,0.25950,0.25225,0.30040,0.3423,0.33510,0.36110,0.0416,...,-0.464686,0.059161,0.352879,0.548867,-0.673300,-0.009437,0.875968,1,3,0


## Resample Dataset

In [4]:
neg_dist = {'Formal settlement': (2/5), 'Unoccupied land': (3/5)}
data = model_utils.resample(data, num_neg_samples=30000, neg_dist=neg_dist, random_state=SEED)

class_dist = data['target'].value_counts().rename(index=value_codes)
area_dist = data['area'].value_counts().rename(index=model_utils.AREA_CODES)

print('Data dimensions: {}'.format(data.shape))
print('\nArea distribution:\n{}'.format(area_dist))
print('\nClass distribution:\n{}'.format(class_dist))
print('\nClass distribution (normalized):\n{}\n'.format(class_dist/len(data)))

data.head()

Data dimensions: (225124, 113)

Area distribution:
Uribia       40390
Riohacha     33501
Cucuta       32665
Arauca       32426
Maicao       30710
Tibu         29027
Arauquita    26405
Name: area, dtype: int64

Class distribution:
Unoccupied land        126000
Formal settlement       77827
Informal settlement     21297
Name: target, dtype: int64

Class distribution (normalized):
Unoccupied land        0.559692
Formal settlement      0.345707
Informal settlement    0.094601
Name: target, dtype: float64



,B1_2015-2016,B2_2015-2016,B3_2015-2016,B4_2015-2016,B5_2015-2016,B6_2015-2016,B7_2015-2016,B8_2015-2016,B9_2015-2016,B10_2015-2016,...,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira,target,area
0,0.1681,0.1326,0.13290,0.13040,0.1571,0.26870,0.32050,0.2784,0.35810,0.0370,...,-0.495702,0.213802,0.122766,0.386654,-0.762164,-0.097429,1.005492,1,2,0
1,0.1933,0.1974,0.19940,0.21950,0.2191,0.26800,0.30120,0.2700,0.32800,0.0384,...,-0.417789,0.102637,0.211351,0.520237,-0.748431,-0.021367,1.030898,1,2,0
2,0.1946,0.1782,0.18280,0.19080,0.2174,0.27470,0.30620,0.2910,0.32340,0.0307,...,-0.188068,0.079984,0.222447,0.746002,-0.641025,-0.030201,1.056083,1,2,0
3,0.1812,0.1497,0.15410,0.15480,0.1868,0.25355,0.29845,0.2618,0.31560,0.0331,...,-0.262399,0.063498,0.280500,0.673429,-0.634472,-0.013013,0.966096,1,2,0
4,0.2069,0.2166,0.23975,0.24575,0.2129,0.26425,0.30840,0.2923,0.33375,0.0384,...,-0.281367,0.067882,0.246190,0.654115,-0.636333,-0.040994,0.967376,1,2,0


## ML Pipeline

In [5]:
from sklearn.linear_model import LogisticRegression

### Model Training & Evaluation

In [6]:
label = 'target'
features = [column  for column in data.columns[:-2]]

data['target'] = data["target"].replace({2:0, 3:0})
print('Class distribution:\n{}'.format(data['target'].value_counts()))

Class distribution:
0    203827
1     21297
Name: target, dtype: int64


### Leave-out-area-out Cross Validation

In [7]:
clf =  LogisticRegression()
model_utils.geospatialcv(data, features, label, clf, verbose=2);


Test set: MAICAO
[[28473  1527]
 [  308   402]]
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     30000
           1       0.21      0.57      0.30       710

    accuracy                           0.94     30710
   macro avg       0.60      0.76      0.64     30710
weighted avg       0.97      0.94      0.95     30710

MAICAO Results: 
- F1 Score: 0.3047
- Kappa Statistics: 0.2803
- Precision: 0.2084
- Recall: 0.5662
- Accuracy: 0.9402

Test set: RIOHACHA
[[29936    64]
 [ 1012  2489]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     30000
           1       0.97      0.71      0.82      3501

    accuracy                           0.97     33501
   macro avg       0.97      0.85      0.90     33501
weighted avg       0.97      0.97      0.97     33501

RIOHACHA Results: 
- F1 Score: 0.8223
- Kappa Statistics: 0.8051
- Precision: 0.9749
- Recall: 0.7109
- Accuracy: 0.9679

Test 

## Train Model

In [8]:
clf = LogisticRegression()
X = data[features].fillna(0)
y = data[label]
clf.fit(X, y);

## Prediction

In [13]:
pd.set_option('use_inf_as_na', True)
area_dict = geoutils.get_filepaths(areas, images_dir, indices_dir, pos_mask_dir, neg_mask_dir)
data = geoutils.read_bands(area_dict, 'uribia')
print('Data dimensions: {}'.format(data.shape))
data.head(3)

100%|██████████| 5/5 [00:25<00:00,  5.06s/it]


Data dimensions: (6217512, 111)


,B1_2015-2016,B2_2015-2016,B3_2015-2016,B4_2015-2016,B5_2015-2016,B6_2015-2016,B7_2015-2016,B8_2015-2016,B9_2015-2016,B10_2015-2016,...,ndbi_2020,savi_2020,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [15]:
preds = clf.predict_proba(data[features].fillna(0))[:, 1]
preds[(data.iloc[:, :-1].sum(axis=1) == 0)] = -1

geoutils.save_predictions(
    preds, 
    image_src=area_dict['uribia']['images'][0], 
    output_file=output_dir+'20200422_uribia.tiff'
)